In [ ]:
from DeepFMKit.core import DeepFitFramework
import matplotlib.pyplot as plt

dff = DeepFitFramework()

# Create a `DeepSimObject` describing an interferometer
label = "my_sim"
dff.new_sim(label)
dff.sims[label].m = 6.0
dff.sims[label].f_mod = 1000
dff.sims[label].f_samp = int(200e3)
dff.sims[label].f_n = 1e6
dff.sims[label].arml_mod_f = 1.0
dff.sims[label].arml_mod_amp = 1e-9
dff.sims[label].arml_mod_n = 1e-12
dff.sims[label].fit_n = 10

# Simulate DFM interferometer with test mass motion and a reference channel
dff.simulate(label, n_seconds=3, simulate="dynamic", ref_channel=False)

# Print system information
dff.sims[label].info()

# NLS fit
dff.fit(label="my_sim", fit_label="my_sim_nls")

In [ ]:
# Run the EKF with default tuning parameters
dff.fit_ekf(label="my_sim", fit_label="my_sim_ekf")

# Run with custom tuning
custom_Q = [1e-9, 1e-9, 1e-7, 1e-7, 1e-9]
custom_R = 0.001
dff.fit_ekf(label="my_sim", fit_label="my_sim_ekf_tuned", Q_diag=custom_Q, R_val=custom_R)

In [ ]:
ax = dff.plot(labels=['my_sim_nls', 'my_sim_ekf', 'my_sim_ekf_tuned'])
ax[0].legend()
plt.show()